# Week 3: Control Theory & Actuation

## Module I: Introduction & Foundations

### Topics Covered

- Drive-by-Wire systems
- Longitudinal Control (PID, Cruise Control)
- Lateral Control (Pure Pursuit, Stanley Method)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Understand drive-by-wire architecture and safety requirements
2. Design and tune PID controllers for longitudinal speed control
3. Implement cruise control and adaptive cruise control (ACC)
4. Apply Pure Pursuit algorithm for path following
5. Implement Stanley method for lateral control
6. Compare different control strategies and tune parameters

---

## Setup

Import required libraries for control systems and visualization

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle, FancyArrow, Rectangle
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Set random seed for reproducibility
np.random.seed(42)

# Plotting configuration
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("Libraries loaded successfully!")
print("NumPy version:", np.__version__)

## 1. Drive-by-Wire Systems

**Drive-by-wire** (DbW) is the use of electronic controls to operate vehicle functions traditionally controlled by mechanical linkages. This technology is essential for autonomous vehicles.

### Traditional vs. Drive-by-Wire

| Function | Traditional (Mechanical) | Drive-by-Wire (Electronic) |
|----------|--------------------------|----------------------------|
| **Steering** | Steering column, rack & pinion | Electric motor + sensors |
| **Throttle** | Cable to throttle body | Electronic throttle control (ETC) |
| **Braking** | Hydraulic master cylinder | Electro-hydraulic brake system |
| **Shifting** | Mechanical shifter linkage | Electronic transmission control |

---

### Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    VEHICLE ECU (Electronic Control Unit)     │
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐      │
│  │  Planning &  │→ │   Control    │→ │  Actuation   │      │
│  │  Perception  │  │  Algorithms  │  │   Commands   │      │
│  └──────────────┘  └──────────────┘  └──────────────┘      │
└─────────────────────────────────────────────────────────────┘
         ↓                    ↓                    ↓
┌─────────────┐      ┌─────────────┐      ┌─────────────┐
│  Steering   │      │  Throttle   │      │    Brake    │
│   Motor     │      │   Actuator  │      │   Actuator  │
└─────────────┘      └─────────────┘      └─────────────┘
         ↓                    ↓                    ↓
┌─────────────┐      ┌─────────────┐      ┌─────────────┐
│  Position   │      │   Throttle  │      │   Brake     │
│   Sensors   │      │   Position  │      │   Pressure  │
└─────────────┘      └─────────────┘      └─────────────┘
         ↓                    ↓                    ↓
    (Feedback)           (Feedback)           (Feedback)
```

---

### Key Components

#### 1. **Steer-by-Wire**
- **Electric motor** applies steering torque
- **Angle sensor** measures steering wheel position
- **Torque sensor** measures driver input
- **Redundant systems** for safety

**Advantages**:
- ✅ Removes steering column (more interior space)
- ✅ Variable steering ratio (parking vs. highway)
- ✅ Enable autonomous steering
- ✅ Better crash safety

**Challenges**:
- ❌ Requires redundant power and computing
- ❌ Must provide steering feel (haptic feedback)
- ❌ Regulatory approval needed

---

#### 2. **Throttle-by-Wire (Electronic Throttle Control)**
- **Accelerator pedal sensor** measures driver input
- **ECU** calculates desired throttle opening
- **Throttle actuator** (DC motor) controls throttle plate
- **Throttle position sensor** provides feedback

**Benefits**:
- Smoother throttle response
- Traction control integration
- Cruise control compatibility
- Better fuel economy

---

#### 3. **Brake-by-Wire**
- **Brake pedal sensor** measures driver input
- **Hydraulic modulator** or **electric calipers** apply braking force
- **Wheel speed sensors** for ABS integration
- **Pressure sensors** for feedback control

**Types**:
- **Electro-Hydraulic Brake (EHB)**: Electronic control, hydraulic actuation
- **Electro-Mechanical Brake (EMB)**: Fully electric calipers (future)

---

### Safety Requirements (ISO 26262)

Drive-by-wire systems must meet **ASIL-D** (Automotive Safety Integrity Level D) - the highest safety level:

1. **Redundancy**: Dual or triple redundant sensors and actuators
2. **Fail-Operational**: System continues operating after single failure
3. **Diagnostics**: Continuous self-monitoring and fault detection
4. **Graceful Degradation**: Safe fallback modes
5. **Watchdog Timers**: Detect software/hardware hangs

**Example**: Steer-by-wire might have:
- 2 steering angle sensors
- 2 motor windings
- 2 independent ECUs
- Mechanical backup connection

---

### Control Loop

Drive-by-wire uses **closed-loop control** with feedback:

```
Reference     ┌──────────┐    Command    ┌──────────┐    Actual
Input    +    │          │               │          │    Output
────────>●───>│Controller├──────────────>│ Actuator ├────────>
         │-   │          │               │          │    │
         │    └──────────┘               └──────────┘    │
         │                                               │
         └───────────────────────────────────────────────┘
                          Feedback
```

**PID Controller** is commonly used for low-level actuation control.

---

### Communication: CAN Bus

**Controller Area Network (CAN)** is the standard for automotive communication:
- **Speed**: 500 kbit/s (CAN 2.0) to 5 Mbit/s (CAN-FD)
- **Topology**: Multi-master bus
- **Reliability**: Error detection, arbitration
- **Latency**: 1-10ms typical

**Example CAN Messages**:
- `0x100`: Steering angle command (10 Hz)
- `0x200`: Throttle position command (20 Hz)
- `0x300`: Brake pressure command (50 Hz)
- `0x400`: Wheel speeds (100 Hz)

In [ ]:
## 2. Longitudinal Control (Speed Control)

**Longitudinal control** manages the vehicle's forward/backward motion by controlling throttle and brake to achieve desired speed or maintain safe following distance.

### PID Controller Fundamentals

The **PID (Proportional-Integral-Derivative) controller** is the workhorse of industrial control systems. It calculates control output based on error between desired and actual values.

**Control Law**:
$$u(t) = K_p \cdot e(t) + K_i \int_0^t e(\tau) d\tau + K_d \frac{de(t)}{dt}$$

Where:
- **u(t)**: Control output (throttle/brake command)
- **e(t) = v_{target} - v_{actual}**: Speed error
- **K_p**: Proportional gain
- **K_i**: Integral gain
- **K_d**: Derivative gain

---

### Three Control Terms

#### 1️⃣ **Proportional (P) Term**
$$u_P(t) = K_p \cdot e(t)$$

**What it does**: React to current error
- Large error → Large control action
- Small error → Small control action

**Effect**:
- ✅ Fast response
- ❌ Steady-state error (never reaches exact target)
- ❌ Can overshoot

**Example**: If car is 10 m/s below target, apply throttle proportional to 10 m/s

---

#### 2️⃣ **Integral (I) Term**
$$u_I(t) = K_i \int_0^t e(\tau) d\tau$$

**What it does**: Accumulate past errors
- Eliminates steady-state error
- "Remembers" persistent errors

**Effect**:
- ✅ Reaches exact target value
- ❌ Can cause overshoot
- ❌ **Integral windup** if saturated

**Example**: If car consistently 1 m/s too slow, integral builds up and increases throttle

---

#### 3️⃣ **Derivative (D) Term**
$$u_D(t) = K_d \frac{de(t)}{dt}$$

**What it does**: Predict future error based on rate of change
- Damping effect
- Reduces overshoot

**Effect**:
- ✅ Reduces oscillations
- ✅ Faster settling
- ❌ Amplifies measurement noise
- ❌ **Derivative kick** on setpoint changes

**Example**: If error is decreasing rapidly, reduce control action to prevent overshoot

---

### Discrete-Time Implementation

For digital control systems (typical automotive ECU runs at 10-100 Hz):

$$\begin{align}
e(k) &= v_{target}(k) - v_{actual}(k) \\
u_P(k) &= K_p \cdot e(k) \\
u_I(k) &= u_I(k-1) + K_i \cdot e(k) \cdot \Delta t \\
u_D(k) &= K_d \cdot \frac{e(k) - e(k-1)}{\Delta t} \\
u(k) &= u_P(k) + u_I(k) + u_D(k)
\end{align}$$

---

### Anti-Windup for Integral Term

**Problem**: When actuator saturates (e.g., throttle at 100%), integral keeps accumulating.

**Solution**: Clamp integral term

```python
# Anti-windup
integral = integral + Ki * error * dt
integral = np.clip(integral, integral_min, integral_max)
```

---

### Derivative Kick Prevention

**Problem**: Sudden setpoint change causes large derivative spike.

**Solution**: Take derivative of **measurement**, not error

```python
# Instead of: derivative = Kd * (error - prev_error) / dt
# Use:
derivative = -Kd * (measurement - prev_measurement) / dt
```

---

### Tuning PID Gains

#### **Ziegler-Nichols Method**
1. Set Ki = Kd = 0
2. Increase Kp until system oscillates
3. Record Ku (ultimate gain) and Pu (oscillation period)
4. Calculate:
   - Kp = 0.6 × Ku
   - Ki = 2 × Kp / Pu
   - Kd = Kp × Pu / 8

#### **Manual Tuning Guidelines**
1. **Start with P-only**: Increase Kp until fast response with acceptable overshoot
2. **Add I**: Increase Ki to eliminate steady-state error (start small!)
3. **Add D**: Increase Kd to reduce overshoot (optional)

**Typical ranges for cruise control**:
- Kp: 100-500 (N per m/s error)
- Ki: 10-50 (N per m error accumulated)
- Kd: 10-100 (N per m/s² rate)

In [ ]:
# PID Controller Implementation for Cruise Control

class PIDController:
    """
    PID controller for longitudinal speed control.
    """
    
    def __init__(self, Kp, Ki, Kd, dt=0.1, output_limits=None, integral_limits=None):
        """
        Args:
            Kp: Proportional gain
            Ki: Integral gain
            Kd: Derivative gain
            dt: Time step (s)
            output_limits: (min, max) tuple for control output
            integral_limits: (min, max) tuple for integral term (anti-windup)
        """
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.dt = dt
        
        self.output_limits = output_limits if output_limits else (-np.inf, np.inf)
        self.integral_limits = integral_limits if integral_limits else (-np.inf, np.inf)
        
        # State variables
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_measurement = 0.0
        
    def update(self, setpoint, measurement, use_derivative_on_measurement=True):
        """
        Calculate PID control output.
        
        Args:
            setpoint: Desired value (target speed)
            measurement: Current value (actual speed)
            use_derivative_on_measurement: If True, take derivative of measurement
                                           (prevents derivative kick)
        
        Returns:
            control_output: PID control signal
        """
        # Calculate error
        error = setpoint - measurement
        
        # Proportional term
        P = self.Kp * error
        
        # Integral term with anti-windup
        self.integral += error * self.dt
        self.integral = np.clip(self.integral, *self.integral_limits)
        I = self.Ki * self.integral
        
        # Derivative term
        if use_derivative_on_measurement:
            # Derivative on measurement (prevents derivative kick)
            D = -self.Kd * (measurement - self.prev_measurement) / self.dt
            self.prev_measurement = measurement
        else:
            # Derivative on error (traditional)
            D = self.Kd * (error - self.prev_error) / self.dt
            self.prev_error = error
        
        # Total control output
        output = P + I + D
        
        # Apply output limits
        output = np.clip(output, *self.output_limits)
        
        return output, P, I, D
    
    def reset(self):
        """Reset controller state."""
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_measurement = 0.0


# Simple vehicle dynamics model
class SimpleLongitudinalVehicle:
    """Simplified vehicle model for longitudinal dynamics."""
    
    def __init__(self, mass=1500, drag_coef=0.3, rolling_resistance=0.01):
        """
        Args:
            mass: Vehicle mass (kg)
            drag_coef: Aerodynamic drag coefficient
            rolling_resistance: Rolling resistance coefficient
        """
        self.mass = mass
        self.drag = drag_coef
        self.rolling_resistance = rolling_resistance
        self.velocity = 0.0  # m/s
        
    def update(self, force, dt=0.1):
        """
        Update vehicle state given applied force.
        
        Args:
            force: Net force (N) - positive for acceleration, negative for braking
            dt: Time step (s)
        """
        # Resistance forces
        drag_force = self.drag * self.velocity**2
        rolling_force = self.rolling_resistance * self.mass * 9.81
        
        # Net force
        net_force = force - drag_force - rolling_force
        
        # Acceleration (F = ma)
        acceleration = net_force / self.mass
        
        # Update velocity
        self.velocity += acceleration * dt
        self.velocity = max(0, self.velocity)  # No negative velocity
        
        return self.velocity


# Simulation: Cruise Control with Different PID Tunings
dt = 0.1
duration = 30  # seconds
time = np.arange(0, duration, dt)

# Target speed profile (step changes)
target_speed = np.ones_like(time) * 20.0  # 20 m/s
target_speed[int(10/dt):] = 25.0  # Increase to 25 m/s at t=10s

# PID configurations to compare
pid_configs = {
    'P-only (Kp=200)': {'Kp': 200, 'Ki': 0, 'Kd': 0},
    'PI (Kp=200, Ki=20)': {'Kp': 200, 'Ki': 20, 'Kd': 0},
    'PID (Kp=200, Ki=20, Kd=50)': {'Kp': 200, 'Ki': 20, 'Kd': 50},
    'Aggressive PID (Kp=400, Ki=40, Kd=100)': {'Kp': 400, 'Ki': 40, 'Kd': 100}
}

results = {}

for name, params in pid_configs.items():
    # Initialize controller and vehicle
    controller = PIDController(
        Kp=params['Kp'], Ki=params['Ki'], Kd=params['Kd'], dt=dt,
        output_limits=(-5000, 5000),  # Force limits (N)
        integral_limits=(-100, 100)   # Anti-windup
    )
    vehicle = SimpleLongitudinalVehicle()
    
    # Storage
    velocities = []
    control_outputs = []
    P_terms = []
    I_terms = []
    D_terms = []
    
    # Simulate
    for t_idx, t in enumerate(time):
        target = target_speed[t_idx]
        current = vehicle.velocity
        
        # PID control
        force, P, I, D = controller.update(target, current)
        
        # Update vehicle
        velocity = vehicle.update(force, dt)
        
        # Record
        velocities.append(velocity)
        control_outputs.append(force)
        P_terms.append(P)
        I_terms.append(I)
        D_terms.append(D)
    
    results[name] = {
        'velocity': np.array(velocities),
        'control': np.array(control_outputs),
        'P': np.array(P_terms),
        'I': np.array(I_terms),
        'D': np.array(D_terms)
    }

# Visualization
fig = plt.figure(figsize=(16, 10))
gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)

# Plot 1: Velocity tracking
ax1 = fig.add_subplot(gs[0, :])
ax1.plot(time, target_speed, 'k--', linewidth=2, label='Target Speed')
for name, data in results.items():
    ax1.plot(time, data['velocity'], linewidth=2, label=name)
ax1.set_xlabel('Time (s)', fontweight='bold')
ax1.set_ylabel('Velocity (m/s)', fontweight='bold')
ax1.set_title('PID Cruise Control: Velocity Tracking', fontweight='bold', fontsize=14)
ax1.legend(loc='lower right')
ax1.grid(True, alpha=0.3)

# Plot 2 & 3: Control effort for two configurations
for idx, (name, data) in enumerate(list(results.items())[:2]):
    ax = fig.add_subplot(gs[1, idx])
    ax.plot(time, data['control'], 'b-', linewidth=2, label='Total')
    ax.plot(time, data['P'], 'r--', linewidth=1.5, label='P term')
    ax.plot(time, data['I'], 'g--', linewidth=1.5, label='I term')
    ax.plot(time, data['D'], 'm--', linewidth=1.5, label='D term')
    ax.set_xlabel('Time (s)', fontweight='bold')
    ax.set_ylabel('Force (N)', fontweight='bold')
    ax.set_title(f'Control Output: {name}', fontweight='bold')
    ax.legend(loc='upper right', fontsize=9)
    ax.grid(True, alpha=0.3)

# Plot 4 & 5: Error analysis for two configurations
for idx, (name, data) in enumerate(list(results.items())[2:4]):
    ax = fig.add_subplot(gs[2, idx])
    error = target_speed - data['velocity']
    ax.plot(time, error, 'r-', linewidth=2)
    ax.axhline(y=0, color='k', linestyle='--', linewidth=1)
    ax.fill_between(time, 0, error, alpha=0.3, color='red')
    ax.set_xlabel('Time (s)', fontweight='bold')
    ax.set_ylabel('Error (m/s)', fontweight='bold')
    ax.set_title(f'Tracking Error: {name}', fontweight='bold')
    ax.grid(True, alpha=0.3)

plt.suptitle('PID Controller Comparison for Cruise Control', fontsize=16, fontweight='bold')
plt.show()

# Performance metrics
print("=" * 80)
print("PERFORMANCE METRICS")
print("=" * 80)
for name, data in results.items():
    error = target_speed - data['velocity']
    mae = np.mean(np.abs(error))
    rmse = np.sqrt(np.mean(error**2))
    overshoot = np.max(data['velocity'] - target_speed)
    
    # Settling time (within 2% of final value)
    final_target = target_speed[-1]
    tolerance = 0.02 * final_target
    settling_idx = np.where(np.abs(data['velocity'][int(10/dt):] - final_target) > tolerance)[0]
    settling_time = (settling_idx[-1] * dt) if len(settling_idx) > 0 else 0
    
    print(f"\n{name}:")
    print(f"  MAE: {mae:.3f} m/s")
    print(f"  RMSE: {rmse:.3f} m/s")
    print(f"  Max Overshoot: {overshoot:.3f} m/s")
    print(f"  Settling Time: {settling_time:.2f} s")
print("=" * 80)

---

## Exercises

*Exercises to be added*

In [ ]:
# Exercise solutions


---

## References

- References to be added